In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
import warnings
warnings.filterwarnings('ignore')

### Regression Analysis

The goal of this workbook was to perform regression modelling on a dataset of information about European tech workers and attempt to predict their compensation based on their attributes and job positions. Most of this work involved the process of data exploration and preparation to ensure the data was in the correct form for regression modelling. I also performed feature selection to try and find the best features to use for modelling and then used grid searching to find the best hyperparameters. While the final results were not as conclusive as I was expecting, the process was still useful to understand how to prepare a complex and real life dataset.

This workbook was originally created as part of my Data Analytics program with NAIT.


### Load Data

In [2]:
# Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Contract duration,Main language at work,Company size,Company type
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5000,30,Full-time employee,Unlimited contract,English,51-100,Product
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.0,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,120000,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,54000.0,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,NaN,29,Full-time employee,Unlimited contract,English,101-1000,Product
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,18/01/2021 15:05:50,31.0,Male,Berlin,Backend Developer,9,5,Senior,Java,"Python, Javascript / Typescript, Java / Scala,...",70000.0,72000,26,Full-time employee,Unlimited contract,English,51-100,Product
1249,18/01/2021 17:46:02,33.0,Male,Berlin,Researcher/ Consumer Insights Analyst,10,1.5,Senior,consumer analysis,NaN,60000.0,3000,unlimited,Full-time employee,Unlimited contract,English,1000+,Product
1250,18/01/2021 23:20:35,39.0,Male,Munich,IT Operations Manager,15,2,Lead,PHP,"Python, C/C++, Javascript / Typescript, Java /...",110000.0,0,28,Full-time employee,Unlimited contract,English,101-1000,eCommerce
1251,19/01/2021 10:17:58,26.0,Male,Saarbrücken,Frontend Developer,7,7,Middle,JavaScript,"Javascript / Typescript, Docker, HTML, CSS; Ad...",38350.0,40950,27,Full-time employee,Unlimited contract,German,101-1000,Product


The data contains many columns that may or may not be useful for regression, but some of them will need to be transformed into an easier to manage format. Since many of them are categorical it will be becassary to deal with them in a particular way to avoid increasing the size of the dataset too much when modelling.

The target variable for this analysis was total compensation, which I will need to create manually based on the information available. 

In [3]:
# Create target variable of total compensation
df["Bonus"] = pd.to_numeric(df["Yearly bonus + stocks in EUR"], downcast="float", errors="coerce")
df["Bonus"].fillna(0, inplace=True)
df["target"] = df["Yearly brutto salary (without bonus and stocks) in EUR"] + df["Bonus"]
df.drop(columns={"Timestamp","Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR", "Bonus"}, inplace=True)

# Remove rows with more than 7 NaN values to clean up data
NaN_threshold = 7
df = df[df.isnull().sum(axis=1) <= NaN_threshold]

### Data Cleaning/Prep

For some of the categorical variables using the mode was a sufficient way to remove the missing values as it would not have much of an effect on the result. However for most of them there were too many different values and attempting to perfrom any encoding would produce far too many new columns. So for the majority of them I changed the least common values to "Other" and assigned them to their own category.

In [4]:
# For example this is a sample of the number of different values in "City"
df['City'].value_counts()

Berlin         677
Munich         235
Frankfurt       44
Hamburg         40
Stuttgart       26
              ... 
Nuremberg        1
Lübeck           1
Malta            1
Dresden          1
Saarbrücken      1
Name: City, Length: 119, dtype: int64

In [8]:
# Finding the mode for "Gender" and "Company size" and using that to fill NaN values in those columns
gender_mode = df['Gender'].mode()[0] 
df.loc[:,'Gender'].fillna(gender_mode, inplace=True)

company_mode = df['Company size'].mode()[0] 
df.loc[:,'Company size'].fillna(company_mode, inplace=True)

# Replacing low frequency values with "Other"
def replace_low_freq(d, col, threshold=10, replacement='other'):
    value_counts = d[col].value_counts() # Specific column 
    to_remove = value_counts[value_counts <= threshold].index
    tmp = d[col].replace(to_replace=to_remove, value=replacement)
    return tmp

# Using low frequency function on each categorical column, then filling NaN values with "Other"
df.loc[:,"Seniority level"] = replace_low_freq(df, "Seniority level", 5, "Other")
df.loc[:,"Seniority level"].fillna("Other", inplace=True)

df.loc[:,"City"] = replace_low_freq(df, "City", 10, "Other")
df.loc[:,"City"].fillna("Other", inplace=True)

df.loc[:,"Your main technology / programming language"] = replace_low_freq(df, "Your main technology / programming language", 10, "Other")
df.loc[:,"Your main technology / programming language"].fillna("Other", inplace=True)

df.loc[:,"Other technologies/programming languages you use often"] = replace_low_freq(df, "Other technologies/programming languages you use often", 10, "Other")
df.loc[:,"Other technologies/programming languages you use often"].fillna("Other", inplace=True)

df.loc[:,"Position "] = replace_low_freq(df, "Position ", 10, "Other")
df.loc[:,"Position "].fillna("Other", inplace=True)

df.loc[:,"Company type"] = replace_low_freq(df, "Company type", 10, "Other")
df.loc[:,"Company type"].fillna("Other", inplace=True)

df.loc[:,"Main language at work"] = replace_low_freq(df, "Main language at work", 10, "Other")
df.loc[:,"Main language at work"].fillna("Other", inplace=True)

df.loc[:,"Employment status"] = replace_low_freq(df, "Employment status", 3, "Other")
df.loc[:,"Employment status"].fillna("Other", inplace=True)

df.loc[:,"Contract duration"] = replace_low_freq(df, "Contract duration", 3, "Other")
df.loc[:,"Contract duration"].fillna("Other", inplace=True)

For the numerical columns the data preparation solution is much more straightforward and mostly involves removing anything that isn't a number and then using the mean of the values to fill the missing values.

In [131]:
# Using an imputer to take the mean of the numerical columns for NaN values
mean_columns = ["Age", "Total years of experience", "Years of experience in Germany", "Number of vacation days"]
imputer = SimpleImputer(strategy='mean')

# Replacing non-numerical symbols in data
replacement = {",":".", "<":" "}
df['Total years of experience'] = df['Total years of experience'].replace(replacement, regex=True)
df['Years of experience in Germany'] = df['Years of experience in Germany'].replace(replacement, regex=True)

# Converting column values to numerical to remove words
df['Total years of experience'] = pd.to_numeric(df['Total years of experience'], errors='coerce')
df['Years of experience in Germany'] = pd.to_numeric(df['Years of experience in Germany'], errors='coerce')
df['Number of vacation days'] = pd.to_numeric(df['Number of vacation days'], errors='coerce')

# Calculate and apply mean of column
df[mean_columns] = imputer.fit_transform(df[mean_columns])

In the case of the "Senority Level" column, the different levels would need to be categorized based on their rank to show their relationship to each other and the other data. 

In [132]:
# Ordinal encoding for "Seniority level"
seniority_order = {'Other': 0, 'Junior': 1, 'Middle': 2, 'Senior': 3, 'Lead': 4, 'Head': 5}
df['Seniority level'] = df['Seniority level'].map(seniority_order)

### Feature Selection

To find the best features out of all the different options I used the feature selection method of variance threshold, which would find the columns that had the least amount of variance compared to the target. This new dataframe could then be used for modelling along with the original data.

In [133]:
# Feature selection using Variance Threshold
df_vf = pd.get_dummies(df, drop_first=True)
y_vf = df_vf["target"]
X_vf = df_vf.drop(columns={"target"})

var_th = VarianceThreshold(.2)

post_vt = var_th.fit_transform(X_vf)
print(post_vt.shape)

mask = var_th.get_support()
new_features = X_vf.columns[mask]
print(new_features)

(1247, 9)
Index(['Age', 'Total years of experience', 'Years of experience in Germany',
       'Seniority level', 'Number of vacation days',
       'Position _Software Engineer',
       'Your main technology / programming language_Other',
       'Company size_101-1000', 'Company type_Product'],
      dtype='object')


In [134]:
# New data frame after results of variance threshold feature selection
df_fs = df.copy()
df_fs.drop(columns={"Gender", "Other technologies/programming languages you use often", "Employment status", "Main language at work", "Contract duration"}, inplace=True)

### Model Testing

Now with the data prepared for modelling in each of the necassary ways I can perform regression analysis to try and predict the target value of total compensation. I used One Hot Encoding on the categorical variables and split the data for training and testing.

In [141]:
# Set up data for model testing
df1 = pd.get_dummies(df, drop_first=True)

y = np.array(df1["target"]).reshape(-1,1)
X = np.array(df1.drop(columns={"target"}))

xTrain,xTest,yTrain,yTest = train_test_split(X,y,test_size=.3)

In [142]:
# Set up data for model testing after feature selection
df_new = pd.get_dummies(df_fs, drop_first=True)

y_fs = np.array(df_new["target"]).reshape(-1,1)
X_fs = np.array(df_new.drop(columns={"target"}))

xTrain_fs,xTest_fs,yTrain_fs,yTest_fs = train_test_split(X_fs,y_fs,test_size=.3)

The regression model I used was a Decision Tree Regressor that was using a grid search to find the best hyperparameters. This waas combined with a pipe that also scaled all the numerical values, which should provide the best set of data for predictions.

In [143]:
#Regression Tree Model

scalar = MinMaxScaler()
regression_tree = DecisionTreeRegressor()
pipe = Pipeline(steps=[('scalar', scalar), ("regression_tree", regression_tree)])

tree_para = {'regression_tree__min_samples_leaf':[4,5,6,7,8],
            'regression_tree__min_samples_split':[4,5,6,7,8],
            'regression_tree__max_depth':[2,3,4,5,6,7],
            'regression_tree__criterion':["friedman_mse", "poisson", "squared_error", "absolute_error"]}

model = GridSearchCV(pipe, param_grid=tree_para, cv=5, n_jobs=-1)
model.fit(xTrain, yTrain)
model_tr = model.best_estimator_


print(model_tr)

print("Train Score with no feature selection:", model.score(xTrain, yTrain))
model_preds = model.predict(xTest)
print("RMSE with no feature selection:", mean_squared_error(yTest,model_preds,squared=False))
print("R2 with no feature selection:", np.mean(cross_val_score(model_tr, xTrain, yTrain.ravel(), cv=5)))



Pipeline(steps=[('scalar', MinMaxScaler()),
                ('regression_tree',
                 DecisionTreeRegressor(criterion='absolute_error', max_depth=3,
                                       min_samples_leaf=7,
                                       min_samples_split=4))])
Train Score with no feature selection: -0.0011589153013702447
RMSE with no feature selection: 38155.93185325165
R2 with no feature selection: 0.09263588866262706


In [144]:
#Regression Tree Model from data after feature selection

scalar_fs = MinMaxScaler()
regression_tree_fs = DecisionTreeRegressor()
pipe_fs = Pipeline(steps=[('scalar', scalar_fs), ("regression_tree", regression_tree_fs)])

tree_para_fs = {'regression_tree__min_samples_leaf':[4,5,6,7,8],
            'regression_tree__min_samples_split':[4,5,6,7,8],
            'regression_tree__max_depth':[2,3,4,5,6,7],
            'regression_tree__criterion':["friedman_mse", "poisson", "squared_error", "absolute_error"]}

model_fs = GridSearchCV(pipe_fs, param_grid=tree_para_fs, cv=5, n_jobs=-1)
model_fs.fit(xTrain_fs, yTrain_fs)
model_tr_fs= model_fs.best_estimator_


print(model_tr)

print("Train Score after feature selection:", model_fs.score(xTrain_fs, yTrain_fs))
model_preds_fs = model_fs.predict(xTest_fs)
print("RMSE after feature selection:", mean_squared_error(yTest_fs,model_preds_fs,squared=False))
print("R2 after feature selection:", np.mean(cross_val_score(model_tr_fs, xTrain_fs, yTrain_fs.ravel(), cv=5)))


Pipeline(steps=[('scalar', MinMaxScaler()),
                ('regression_tree',
                 DecisionTreeRegressor(criterion='absolute_error', max_depth=3,
                                       min_samples_leaf=7,
                                       min_samples_split=4))])
Train Score after feature selection: 0.05729711774014179
RMSE after feature selection: 284015832.4165602
R2 after feature selection: 0.05522113574581004


The results from these regression models are quite low in terms of how accurately they can predict the value of total compensation, as indicated by the results above. They seem to indicate that there is some issue with the data and the way it was prepared or perhaps the data is too varied in terms of the different columns and what they represent. This could be the nature of using more complex and real data that requires a lot of fine tuning to be more useful when it comes to modeling.

Overall I feel this notebook was useful to demonstrate the data preparation process and how I come about finding a solution, which is a fundamental part of the analytical process.